In [1]:
%pylab inline
import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd
import asyncio
from pyppeteer import launch
import nest_asyncio
# We need to use pyppeteer and asyncio to get the data from the sota pages as they are js generated
import time

Populating the interactive namespace from numpy and matplotlib


In [2]:
# SOTA
url="https://www.sotadata.org.uk/summitslist.csv"

dtype={"GridRef1": "string", "GridRef2": "string"}
names=["SummitCode", "AssociationName", "RegionName", "SummitName", 
       "AltM", "AltFt", "GridRef1", "GridRef2", "Longitude", "Latitude", "Points",
       "BonusPoints", "ValidFrom", "ValidTo", "ActivationCount", "ActivationDate",
       "ActivationCall"]

sotadf=pd.read_csv(url, skiprows=2, names=names, dtype=dtype)
sotadf = sotadf[sotadf.AssociationName == "Ireland"]

In [91]:
summits = sotadf['SummitCode']
try:
    df = pd.read_json("summit-activation-histories")
except:
    df = pd.DataFrame()


In [ ]:
try:
    i = len(df['Summit'].unique()) - 1
except:
    i = 0
    
#i=302
nest_asyncio.apply()

async def main(summit):
    browser = await launch(headless=False)
    page = await browser.newPage()
    await page.goto('https://summits.sota.org.uk/activity/' + summit,{
        'waitUntil': 'networkidle0'
    })
    content = await page.evaluate('document.body.textContent', force_expr=True)
    content = await page.content()
    try:
        df = pd.read_html(content)
    except:
        df = None
    await browser.close()
    return df
    
for summit in summits[i:]:
    retries = 0
    while retries < 10:
        try:
            d = asyncio.get_event_loop().run_until_complete(main(summit))
            break;
        except:
            retries = retries + 1
            time.sleep(2)
            d = asyncio.get_event_loop().run_until_complete(main(summit))
    try:
        d[0]['Summit'] = summit
        d[0]['Date']= pd.to_datetime(d[0]['Date'])
        df = df.append(d[0])
    except:
        pass
    #print(str(i) + summit)
    i = i + 1
    time.sleep(2)
    
df.reset_index(inplace=True)
df.to_json("summit-activation-histories")

In [82]:
df.reset_index(inplace=True, drop=True)
df.to_json("summit-activation-histories")
df = pd.read_json("summit-activation-histories")
df

,index,Date,Activator,QSOs,Summit
0,0,2019-12-29,EI7KP/P,10,None
1,1,2018-11-11,EI7KP/P,15,None
2,2,2015-12-27,EI/GI4ONL/P,19,None
3,3,2015-12-27,EI/MI0JST/P,9,None
4,4,2012-04-22,EI/GI4ONL/P,27,None
...,...,...,...,...,...
2042,0,2014-10-20,EI/ON4TA/P,23,EI/IW-085
2043,1,2012-06-28,EI/M0JDK/P,19,EI/IW-085
2044,0,2016-07-29,EJ7GEB/P,38,EI/IW-086
2045,1,2012-03-02,EJ/ON4TA/P,73,EI/IW-086


In [ ]:
df = df.drop(["index"], axis=1)

# What's the most number of summits activated in one day?

In [93]:
df = df.drop_duplicates()
df.groupby([df.Date,df.Activator]).count().sort_values("Summit", ascending=False).head(40).drop("QSOs",axis=1)




,,Summit
Date,Activator,
2010-08-24,EI/G4WSB/P,6
2010-08-26,EI/G4WSB/P,5
2010-08-21,EI/G4WSB/P,5
2010-08-22,EI/G4WSB/P,5
2010-08-23,EI/G4WSB/P,5
2010-09-02,EI/G4WSB/P,4
2010-08-27,EI/G4WSB/P,4
2013-07-19,EI/MM0YCJ/P,4
2013-07-12,EI/MM0YCJ/P,3
